# Command Pattern

The Command pattern encapsulates a request as an object, thereby allowing for parameterization of clients with different requests, queuing of requests, and support for undoable operations.

## Problem

You need to decouple an object that invokes an operation from the provider that actually performs it, or you need to parameterize objects with operations, support undoable actions, or create queues of operations.

## Solution

Create a command class that encapsulates all information needed to perform an action, including the method to call, the method arguments, and the object that implements the method. This allows for decoupling and flexibility in executing operations.

In [1]:
from abc import ABC, abstractmethod


# Command interface
class Command(ABC):
    @abstractmethod
    def execute(self) -> None:
        pass

    @abstractmethod
    def undo(self) -> None:
        pass

In [2]:
# Receiver class
class Editor:
    def __init__(self):
        self.text = ""

    def write(self, text):
        self.text += text
        print(f"Editor: Added text '{text}'")
        print(f"Current text: '{self.text}'")

    def delete_last(self, length):
        deleted_text = self.text[-length:]
        self.text = self.text[:-length]
        print(f"Editor: Deleted text '{deleted_text}'")
        print(f"Current text: '{self.text}'")
        return deleted_text

In [3]:
# Concrete Commands
class WriteCommand(Command):
    def __init__(self, editor: Editor, text: str):
        self.editor = editor
        self.text = text

    def execute(self) -> None:
        self.editor.write(self.text)

    def undo(self) -> None:
        self.editor.delete_last(len(self.text))


class DeleteCommand(Command):
    def __init__(self, editor: Editor, length: int):
        self.editor = editor
        self.length = length
        self.backup = ""

    def execute(self) -> None:
        self.backup = self.editor.delete_last(self.length)

    def undo(self) -> None:
        self.editor.write(self.backup)

In [4]:
# Invoker
class CommandHistory:
    def __init__(self):
        self.history = []

    def push(self, command: Command):
        self.history.append(command)

    def pop(self) -> Command:
        return self.history.pop() if self.history else None


class Application:
    def __init__(self):
        self.editor = Editor()
        self.history = CommandHistory()

    def execute_command(self, command: Command):
        command.execute()
        self.history.push(command)

    def undo(self):
        command = self.history.pop()
        if command:
            print("Undoing the last command...")
            command.undo()
        else:
            print("Nothing to undo.")

In [5]:
# Client code
app = Application()

# Execute write command
app.execute_command(WriteCommand(app.editor, "Hello, "))

# Execute another write command
app.execute_command(WriteCommand(app.editor, "World!"))

# Execute delete command
app.execute_command(DeleteCommand(app.editor, 6))

# Undo last command (delete)
app.undo()

# Undo previous command (write "World!")
app.undo()

Editor: Added text 'Hello, '
Current text: 'Hello, '
Editor: Added text 'World!'
Current text: 'Hello, World!'
Editor: Deleted text 'World!'
Current text: 'Hello, '
Undoing the last command...
Editor: Added text 'World!'
Current text: 'Hello, World!'
Undoing the last command...
Editor: Deleted text 'World!'
Current text: 'Hello, '


## Benefits

* **Decoupling**: Separates the object that invokes the operation from the one that performs it
* **Extensibility**: New commands can be added without changing existing code
* **Composite Commands**: Commands can be combined to create macro commands
* **Undo/Redo Support**: Commands can store state for reversing their effects
* **Deferred Execution**: Commands can be scheduled, queued, or executed remotely